In [1]:
import base64
from PIL import Image
from io import BytesIO
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def encode_image(image):
    buffered = BytesIO()
    w,h = image.size
    if w>512 or h>512:
        scale = 512 / max(w,h)
    else:
        scale = 1.0
    resize_im = image.resize((int(w*scale),int(h*scale)))
    resize_im.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    return img_str

In [3]:
# 이미지와 프롬프트를 받아서 OpenAI API에 요청을 보내는 함수
def img_captioning(prompt, image):
    
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {
                "role": "user",
                "content":[
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{encode_image(image)}"
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )
    return response.choices[0].message.content


In [4]:
prompt1 = f"""
영상에서 무슨 일이 일어나고 있는지 설명해줘.
빨간색 유니폼을 입은 선수들의 동작에 집중해서 각각 어떻게 움직이고 있는지 설명해줘.
한 줄에 한문장씩 출력해줘.
"""

prompt2 = f"""
영상에서 무슨 일이 일어나고 있는지 설명해줘.
파란색 유니폼을 입은 선수의 동작에 집중해서 왜 저런 동작을 했고 앞으로 무슨일이 일어날지 포함해서 설명해줘.
한 줄에 한문장씩 출력해줘.
"""

img = Image.open('images/06.jpg')
response = img_captioning(prompt2, img)
print(response)



파란색 유니폼을 입은 선수는 공을 가로채기 위해 슬라이딩 태클을 시도하고 있습니다.   
이 동작은 상대 선수의 공 소유를 방해하려는 의도입니다.   
슬라이딩 태클 이후, 파란색 유니폼 선수는 공을 소유하려고 할 것입니다.   
상대편은 이를 피하거나 넘어지지 않으려는 시도를 할 수 있습니다.  
게임은 이 상황의 결과에 따라 계속 진행될 것입니다.
